# Machine Learning with SAP Datasphere, Hands-On Workshop
## Explore and prepare the data in SAP Datasphere

Retrieve the credentials to connect to SAP Datasphere

In [ ]:
import json
file = open('credentials.json', 'r')
credentials = json.load(file)
file.close()

Establish a connection with SAP Datasphere. <font color='red'>Ensure that you updated the credentials.json file with the details of the Database User that you created earlier on.</font>

In [ ]:
import hana_ml.dataframe as dataframe
conn = dataframe.ConnectionContext(address  = credentials['hana_address'],
                                   port     = credentials['hana_port'], 
                                   user     = credentials['hana_user'], 
                                   password = credentials['hana_password'], 
                                  )
conn.connection.isconnected()

Point a hana_ml DataFrame to the view in SAP Datasphere, which returns the data that was loaded in the  previous Notebook. The variable df_remote is a reference to the data, which remains in SAP Datasphere. This command does not extract the data from SAP Datasphere. We will use such hana_ml Dataframes extensively in this workshop as they allow to process the data in SAP Datasphere. <font color='red'>Ensure to change the schema paramter to your Datasphere user, ie schema='AC54952UXX'</font>

In [ ]:
df_remote = conn.table('V_LucerneElectricity', schema='DSP_ML_HANDSON')

Retrieve and display a few rows of data from SAP Datasphere. First restrict the hana_ml Dataframe to only 5 rows with the head()-method. Then download these 5 rows with the collect()-method into the Notebook.

In [ ]:
df_remote.head(5).collect()

Get a first overview of the data

In [ ]:
df_remote.describe().collect()

All above statistics were calculated in SAP Datasphere. See the SELECT Statement that was created by the describe()-method and executed in SAP Datasphere.

In [ ]:
print(df_remote.describe().select_statement)

Retrieve and plot the most recent 100 data points (each data point corresponds to a 15-minute interval)

In [ ]:
df_data = df_remote.sort('TIMESTAMP').tail(n=100, ref_col='TIMESTAMP').collect()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
sns.set(rc={'figure.figsize':(15,5)})
sns.lineplot(data=df_data, x="TIMESTAMP", y="CONSUMPTION")
plt.xticks(rotation=45);

Add a column that standardises the TIMESTAMP column to the full hour, so  minutes 15, 30 and 45 are set to 0. This is done by subtracting the seconds that passed since the full hour. This column is part of the hana_ml Dataframe. The underlying table remains unchanged.

In [ ]:
df_remote = df_remote.select('*', ("ADD_SECONDS(TIMESTAMP, -MINUTE(TIMESTAMP)*60)", 'DATEHOUR'))
df_remote.head(5).collect()

Use the newly created column to aggregate the consumtion on the hour of the day, thereby adding the consumption from minutes 15, 30 and 45 to the corresponding full hour. This aggregation is done on the fly. The aggregate is not persisted as physical table.

In [ ]:
df_rem_agg = df_remote.agg([('sum', 'CONSUMPTION', 'CONSUMPTION_H')], group_by='DATEHOUR').sort('DATEHOUR')
df_rem_agg.head(5).collect()

Display the SELECT statement that is underpinning the hana_ml DataFrame df_rem_agg. Notice the sum("CONSUMPTION"), which creates the aggregation.

In [ ]:
print(df_rem_agg.select_statement)

Add column that shows only the hour of the day. We use this column for further data analyis.

In [ ]:
df_rem_agg = df_rem_agg.select('*', ('HOUR(DATEHOUR)', 'HOUR'))
df_rem_agg.head(5).collect()

See the distribution of consumption by the hour of the day. The creation of this chart might take a few seconds

In [ ]:
import matplotlib.pyplot as plt
from hana_ml.visualizers.eda import EDAVisualizer
f = plt.figure()
ax1 = f.add_subplot(111) # 111 refers to 1x1 grid, 1st subplot
eda = EDAVisualizer(ax1)
ax, cont = eda.box_plot(data=df_rem_agg, column='CONSUMPTION_H', groupby='HOUR', vert=True, outliers=False)

See the SELECT Statement that is behind the current hana_ml DataFrame

In [ ]:
print(df_rem_agg.select_statement)

Save the data structure, which includes the two calculated columns DATEHOUR and HOUR, as view for use in the next notebook. Only the semantics are saved, the data is not duplicated.

In [ ]:
df_rem_agg.save('V2_LUCERNEELECTRICITY', table_type='VIEW', force=True)